In [1]:
import tweepy
import numpy as np
import pandas as pd
import datetime
import pytz

In [2]:
consumer_key = "6CssxvG8WIAk7u7TBRpbScGhN" #Your API/Consumer key 
consumer_secret = "SKvi5CEXG6oNZkNaqZZZehc7ZZ8l2GGkpafLTkViLHvMutYcww" #Your API/Consumer Secret Key
access_token = "1315079054366666752-xG1v0KJBCFu1nMNmHzW5BTouDCEvAW"    #Your Access token key
access_token_secret = "ZILIStmHT5LsbMM84pDuiCLSAX8BODu9FhrBW152lr7Ij" #Your Access token Secret key

#Pass in our twitter API authentication key
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret,
    access_token, access_token_secret
)

#Instantiate the tweepy API
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
username = "farrelium"

# Define tweets will be taken from 2 years from now
utc = pytz.UTC
endDate = datetime.datetime.now()
startDate = endDate - datetime.timedelta(days=2*365)
endDate = utc.localize(endDate) 
startDate = utc.localize(startDate)

tweets = []

try:
    # The number of tweets we want to retrieved from the user in a specific period of time
    temp_tweets = api.user_timeline(screen_name=username)
    for tweet in temp_tweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)
            
    while (temp_tweets[-1].created_at > startDate):
        temp_tweets = api.user_timeline(screen_name=username, max_id = temp_tweets[-1].id)
        for tweet in temp_tweets:
            if tweet.created_at < endDate and tweet.created_at > startDate and len(tweets) < 200: # Max 200 tweets
                tweets.append(tweet)
            
    #Pulling Some attributes from the tweet
    #attributes_container = [[tweet.created_at,tweet.favorite_count,tweet.source,tweet.text] for tweet in tweets]
    attributes_container = [[tweet.created_at,tweet.text] for tweet in tweets]

    #Creation of column list to rename the columns in the dataframe
    #columns = ["Date Created", "Number of Likes", "Source of Tweet", "Tweet"]
    columns = ["date_created", "tweet"]
    
    #Creation of Dataframe
    crawl_df = pd.DataFrame(attributes_container, columns=columns)
except BaseException as e:
    # Kalau error, return e (di flask)
    print('Status Failed On,',str(e))

Status Failed On, 401 Unauthorized
89 - Invalid or expired token.


In [4]:
tweets[0].text

'RT @ngePopMie: Karna Mamang Pop Mie tugasnya bikin puasa kalian lebih seru dan manteb, gw buka challenge aja deh nih buat antum-antum semwa…'

In [5]:
attributes_container

[[datetime.datetime(2023, 4, 3, 13, 46, 10, tzinfo=datetime.timezone.utc),
  'RT @ngePopMie: Karna Mamang Pop Mie tugasnya bikin puasa kalian lebih seru dan manteb, gw buka challenge aja deh nih buat antum-antum semwa…'],
 [datetime.datetime(2023, 4, 1, 6, 38, 47, tzinfo=datetime.timezone.utc),
  '@surrealnoise Hamdalah ditanya ‘followers’ teh apa?'],
 [datetime.datetime(2023, 4, 1, 6, 12, 6, tzinfo=datetime.timezone.utc),
  'RT @Midjan_La_2: Sekelompok pemuda berseragam loreng oranye mirip salah satu ormas pemuda di Medan, menggeruduk ruang Direktur Rumah Sakit…'],
 [datetime.datetime(2023, 4, 1, 6, 11, 13, tzinfo=datetime.timezone.utc),
  'RT @investorgabut: Ironisnya kenaikan jumlah wajib pajak yg lapor SPT dianggap sebagai prestasi oleh DJP, bukan prestasi masyarakat Indones…'],
 [datetime.datetime(2023, 4, 1, 6, 10, 19, tzinfo=datetime.timezone.utc),
  '🤣🤣🤣🤣🤣🤣🤣\nWalaupun mungkin konteksnya bukan minta gratisan ya, tapi nunjukin followers saya segini udah cukup funny l… https://t.c

In [6]:
crawl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   date_created  200 non-null    datetime64[ns, UTC]
 1   tweet         200 non-null    object             
dtypes: datetime64[ns, UTC](1), object(1)
memory usage: 3.2+ KB


In [7]:
crawl_df.tail()

,date_created,tweet
195,2022-12-09 21:59:24+00:00,RT @andreasharsono: Majelis hakim di Makassar ...
196,2022-12-09 21:59:24+00:00,RT @andreasharsono: Majelis hakim di Makassar ...
197,2022-12-06 23:34:32+00:00,RT @__dtm__: Kemana analogi indah ttg satpam d...
198,2022-12-06 14:05:56+00:00,RT @PolJokesID: Mungkin ini saatnya kita lebih...
199,2022-12-06 06:08:55+00:00,RT @PolJokesID: Ada baiknya MK ini dibubarkan ...


In [8]:
len(crawl_df)

200

In [9]:
# Check if there's min 20 tweets
if len(crawl_df) > 20:
    sample_df = crawl_df.sample(n=20)
else:
    sample_df = crawl_df.copy()
    
sample_df

,date_created,tweet
44,2023-03-08 01:52:24+00:00,RT @MrBekalicky89: Salut dan dukung pernyataan...
141,2022-12-24 07:30:12+00:00,RT @mouldie_sep: Wkwkwkwk mau muter balik nyal...
65,2023-02-19 03:28:55+00:00,RT @Heraloebss: Tahanan BNN Makale : Kami dili...
13,2023-03-22 10:20:17+00:00,RT @RajaJuliAntoni: Kepala BPN Jaktim Sudarman...
162,2022-12-18 18:45:45+00:00,RT @ainurohman: Bermula dari cita2 masa kecil ...
120,2023-01-05 21:52:12+00:00,RT @evimsofian: Of all the people you can idol...
31,2023-03-11 02:15:37+00:00,"RT @TarizSolis: Hati remuk, air mata tumpah &a..."
62,2023-02-22 02:26:07+00:00,RT @detikcom: Bandar sabu di Toraja Utara meng...
182,2022-12-14 14:39:18+00:00,RT @KompasTV: Ahli Poligraf: Putri Candrawathi...
79,2023-02-05 02:20:01+00:00,@ArditErwandha @erwinwu_ @Standupindo Fak


## Preprocessing

In [10]:
import re
import preprocessor
import nltk
import goslate
import enchant
from googletrans import Translator
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

en = enchant.Dict("en_US")
idn = []

In [13]:
def casefolding(review):
    review = review.lower()
    return review

def filtering(review):
    review = re.sub(r'@[^\s]+', '', review)  # @username
    review = re.sub(r'#([^\s]+)', '', review)  # hashtag
    review = re.sub(r'https:[^\s]+', '', review)  # URL links
    review = re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", review)  # symbol, char
    review = re.sub(r'[^\x00-\x7f]+', '', review)  # non ASCII chars
    review = re.sub(r'\s+', ' ', review)  # duplicate whitespace
    return preprocessor.clean(review)

def tokenizing(review):
    token = nltk.word_tokenize(review)
    return token

def stemming(review):
    factorySt = StemmerFactory()
    stemmer = factorySt.create_stemmer()
    review = stemmer.stem(review)
    return review

def stopWordRemoving(review):
    factorySw = StopWordRemoverFactory()
    stopword = factorySw.create_stop_word_remover()
    review = stopword.remove(review)
    return review

def slangWordConverting(review):
    slangwords = dict()
    with open('slangword-id.txt') as wordfile:
        for word in wordfile:
            word = word.split('=')
            slangwords[word[0]] = word[1].replace('\n', '')

    wordsArray, fixed = review.split(' '), []
    for word in wordsArray:
        if word in slangwords:
            word = slangwords[word]
        fixed.append(word)
        review = ' '.join(fixed)
    return review

def characterRepeating(review):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    tempWord = ''
    for word in review.split(' '):
        if word != '':
            if en.check(word):
                tempWord += word+' '
            elif word in idn:
                tempWord += word+' '
            else:
                tempWord += pattern.sub(r"\1", word) + ' '
    return tempWord


def translating(review):
    gs = goslate.Goslate()
    translatedText = gs.translate(review, 'id')
    return translatedText

def translating2(review):
    translator = Translator()
    translations = translator.translate(review, dest='id')
    return translations.text

def preprocess_tweets(tweet):
    tweet = filtering(str(tweet))
    tweet = casefolding(str(tweet))
    tweet = characterRepeating(str(tweet))
    # tweet = translating(str(tweet))
    tweet = stemming(str(tweet))
    tweet = stopWordRemoving(str(tweet))
    tweet = slangWordConverting(str(tweet))
    # print('Processed: '+str(tweet) + '\n')
    return tweet

In [14]:
sample_df['preprocessed'] = sample_df['tweet'].apply(preprocess_tweets)

In [15]:
sample_df['preprocessed']

44     hormat dukung pernyatan saudara laki uprit kri...
141    tawa mau putar balik nyalip truk kiri kena til...
65                        tahan bn makale lindung polres
13     kepala bpn jaktim sudarman harjasaputra copot ...
162    mula cita masa kecil sederhana bagai guru olah...
120    dari semua itu orang anda bisa idolize kena t ...
31     hati remuk air mata tumpah sampai badan getar ...
62     bandar sabu toraja utara saya beking polisi ke...
182    ahli poligraf putri candrawathi raih skor indi...
79                                                bajing
142     layan indonesia polisi tidak pakai seragam doang
154    id cinta untuk dengar fifas jelas dari kenapa ...
183          orang memang mahir angkat beban jaman jajah
91     bela tumpang kena catcalling kenapa malah kero...
108    ibu mega pidato membangakan tasdi mantan sopir...
101                                             dis diam
97     anak dititipin seni bebi sitter hari peduli pe...
15                             

In [16]:
result_tweet = " ".join(sample_df['preprocessed'])

In [17]:
result_df = pd.DataFrame({
    'username': [username],
    'tweet': [result_tweet]
})

result_df

,username,tweet
0,bintangemon,hormat dukung pernyatan saudara laki uprit kri...


## Inference

In [18]:
import joblib
import pickle

# Load tfidf
with open("../personality-evaluator/PPT-09/models/tfidf_2.pickle", "rb") as file:
    tfidf = pickle.load(file)
    
# Load best model
model = joblib.load("../personality-evaluator/PPT-09/models/pipeline.joblib")

In [19]:
X = result_df['tweet']

In [20]:
X

0    hormat dukung pernyatan saudara laki uprit kri...
Name: tweet, dtype: object

In [21]:
y_pred_proba = model.predict_proba(X)
y_pred_proba

array([[0.152, 0.092, 0.368, 0.312, 0.076]])

In [22]:
list_proba = y_pred_proba[0]
list_class = [0, 1, 2, 3, 4]
list_username = [username] * 5

In [23]:
reshape_df = pd.DataFrame(list(zip(list_username, list_class, list_proba)),
               columns =['user', 'class', 'proba_tweet'])
reshape_df

,user,class,proba_tweet
0,bintangemon,0,0.152
1,bintangemon,1,0.092
2,bintangemon,2,0.368
3,bintangemon,3,0.312
4,bintangemon,4,0.076


In [25]:
reshape_df.iloc[0]['proba_tweet']

0.152

In [ ]:
# Ecek-eceknya dapet data dari frontend berupa json
request = {
    'test_result': [3.75, 2.22, 3.56, 5.00, 6.0]
}

reshape_df['test_result'] = request['test_result']
reshape_df['proba_tweet'] = reshape_df['proba_tweet'] * 10 # Normalisasi
reshape_df

In [23]:
reshape_df['inference'] = reshape_df['proba_tweet']*0.6 + reshape_df['test_result']*0.4

In [24]:
reshape_df.head()

,user,class,proba_tweet,test_result,inference
0,bintangemon,0,1.360000,3.75,2.316000
1,bintangemon,1,0.720000,2.22,1.320000
2,bintangemon,2,3.567619,3.56,3.564571
3,bintangemon,3,3.432381,5.00,4.059429
4,bintangemon,4,0.920000,6.00,2.952000


In [25]:
y = reshape_df['inference'].argmax()

In [26]:
result = reshape_df.iloc[y]
result

user           bintangemon
class                    3
proba_tweet       3.432381
test_result            5.0
inference         4.059429
Name: 3, dtype: object

In [27]:
d = {
     0:'Neuroticism',
     1:'Agreeableness',
     2:'Extraversion',
     3:'Openness',
     4:'Conscientiousness'
}

# Return class
d[result['class']]

'Openness'

## Labeling BFI

In [35]:
OCEANnumber = {
    'Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
    'Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
    'Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
    'Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
    'Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
}

R = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]

In [36]:
OCEAN = {
        'Extraversion': 0,
        'Agreeableness': 0,
        'Conscientiousness': 0,
        'Neuroticism': 0,
        'Openness': 0
    }

bfi_form =  ["h3llatrash", 2, 4, 5, 5, 4, 4, 3, 2, 2, 4, 3, 3, 4, 4, 5, 3, 3, 4, 5, 5, 4, 2, 2, 3, 4, 4, 5, 4, 4, 5, 5, 1, 2, 3, 4, 2, 4, 4, 5, 5, 1, 3, 4, 4]

In [37]:
for nomer_soal in range(1, 45):
    if nomer_soal in OCEANnumber['Extraversion']:
            if int(bfi_form[nomer_soal]) > 3:
                if nomer_soal not in R:
                    OCEAN['Extraversion'] += 1
                    
for nomer_soal in range(1, 45):
    if nomer_soal in OCEANnumber['Agreeableness']:
            if int(bfi_form[nomer_soal]) > 3:
                if nomer_soal not in R:
                    OCEAN['Agreeableness'] += 1
                    
for nomer_soal in range(1, 45):
    if nomer_soal in OCEANnumber['Conscientiousness']:
            if int(bfi_form[nomer_soal]) > 3:
                if nomer_soal not in R:
                    OCEAN['Conscientiousness'] += 1
                    
for nomer_soal in range(1, 45):
    if nomer_soal in OCEANnumber['Neuroticism']:
            if int(bfi_form[nomer_soal]) > 3:
                if nomer_soal not in R:
                    OCEAN['Neuroticism'] += 1
                    
for nomer_soal in range(1, 45):
    if nomer_soal in OCEANnumber['Openness']:
            if int(bfi_form[nomer_soal]) > 3:
                if nomer_soal not in R:
                    OCEAN['Openness'] += 1

In [38]:
OCEAN

{'Extraversion': 1,
 'Agreeableness': 0,
 'Conscientiousness': 4,
 'Neuroticism': 5,
 'Openness': 8}

In [39]:
print(len(OCEANnumber['Extraversion']))
print(len(OCEANnumber['Agreeableness']))
print(len(OCEANnumber['Conscientiousness']))
print(len(OCEANnumber['Neuroticism']))
print(len(OCEANnumber['Openness']))

8
9
9
8
10


In [40]:
OCEAN['Extraversion'] /= len(OCEANnumber['Extraversion']) / 10
OCEAN['Agreeableness'] /= len(OCEANnumber['Agreeableness']) / 10
OCEAN['Conscientiousness'] /= len(OCEANnumber['Conscientiousness']) / 10
OCEAN['Neuroticism'] /= len(OCEANnumber['Neuroticism']) / 10
OCEAN['Openness'] /= len(OCEANnumber['Openness']) / 10

In [41]:
OCEAN['Extraversion'] = round(OCEAN['Extraversion'], 2)
OCEAN['Agreeableness'] = round(OCEAN['Agreeableness'], 2)
OCEAN['Conscientiousness'] = round(OCEAN['Conscientiousness'], 2)
OCEAN['Neuroticism'] = round(OCEAN['Neuroticism'], 2)
OCEAN['Openness'] = round(OCEAN['Openness'], 2)

OCEAN['Extraversion'] = str(OCEAN['Extraversion'])
OCEAN['Agreeableness'] = str(OCEAN['Agreeableness'])
OCEAN['Conscientiousness'] = str(OCEAN['Conscientiousness'])
OCEAN['Neuroticism'] = str(OCEAN['Neuroticism'])
OCEAN['Openness'] = str(OCEAN['Openness'])

In [42]:
OCEAN

{'Extraversion': '1.25',
 'Agreeableness': '0.0',
 'Conscientiousness': '4.44',
 'Neuroticism': '6.25',
 'Openness': '8.0'}